In [ ]:
!pip install transformers

     |████████████████████████████████| 665kB 2.6MB/s 
     |████████████████████████████████| 890kB 12.8MB/s 
     |████████████████████████████████| 1.1MB 16.9MB/s 
     |████████████████████████████████| 3.8MB 21.3MB/s 
  Created wheel for sacremoses: filename=sacremoses-0.0.43-cp36-none-any.whl size=893260 sha256=dc08dbd9d3e9cbb9bdd2c51ce222a4105dd512030a6dddb1fed829d82397dba5
  Stored in directory: /root/.cache/pip/wheels/29/3c/fd/7ce5c3f0666dab31a50123635e6fb5e19ceb42ce38d4e58f45
Successfully built sacremoses


In [ ]:
import csv
import re
import random
import torch
import pickle
from torch.utils.data import Dataset
from torch import nn
from transformers import BertTokenizer, BertModel
from torch.utils.tensorboard import SummaryWriter
from shutil import copy2
import os
import subprocess
import numpy as np


In [ ]:
tokenizer = BertTokenizer.from_pretrained('bert-base-cased')
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [ ]:
!unzip data.zip

Archive:  data.zip
   creating: data/
  inflating: data/glove.6B.100d.txt  
   creating: data/task-1/
  inflating: data/task-1/.DS_Store   
  inflating: data/task-1/dev.csv     
  inflating: data/task-1/train.csv   
  inflating: data/task-1/test.csv    
  inflating: data/.DS_Store          
   creating: data/task-2/
  inflating: data/task-2/.DS_Store   
  inflating: data/task-2/dev.csv     
  inflating: data/task-2/train.csv   
  inflating: data/task-2/test.csv    


In [ ]:
print('Indexing word vectors.')

embeddings_index = {}
f = open('data/glove.6B.100d.txt', encoding='utf-8')
for line in f:
    values = line.split()
    word = values[0]
    coefs = np.asarray(values[1:], dtype='float32')
    embeddings_index[word] = coefs
f.close()

Indexing word vectors.


In [ ]:
class HeadlineDataset(Dataset):
    def __init__(self, dataset = 'training',task = 'task-1'):
        assert dataset in ('training', 'dev', 'test')
        self.dataset = dataset
        self.task = task
        dataset_path = f'./data/{task}/{dataset}.csv'
        with open(dataset_path) as f:
            reader = csv.DictReader(f)
            self.rows = list(reader)
        self.glove_embeddings = embeddings_index

    
    def get_edited_text(self,row):
      if(self.task == 'task-1'):
        orig = row['original']
        edited = re.sub(r"<.+/>", row['edit'], orig)
        return edited
      else: 
        orig = row['original1']
        edited_1 = re.sub(r"<.+/>", row['edit1'], orig)
        edited_2 = re.sub(r"<.+/>", row['edit2'], orig)
        return edited_1,edited_2


     
    def get_orig_text(self,row):
      if(self.task == 'task-1'):
        orig = row['original']
        edited = re.sub(r"<(.+)/>", r"\g<1>", orig)
        return edited
      else:
        orig = row['original1']
        edited = re.sub(r"<(.+)/>", r"\g<1>", orig)
        return edited

     
    def get_orig_phrase(self,row):
      if(self.task == 'task-1'):
        orig = row['original']
        return re.match(r".*?<(.+)/>.*", orig).group(1).strip()
      else:
        orig = row['original1']
        return re.match(r".*?<(.+)/>.*", orig).group(1).strip()

    def __len__(self):
        return len(self.rows)

    def __getitem__(self, idx):
      if(self.task=='task-1'):
        row = self.rows[idx]
        headline = self.get_edited_text(self.rows[idx])
        word_vectors = []
        for word in headline.split():
            word_vectors.append(torch.from_numpy(self.glove_embeddings.get(word, self.glove_embeddings['unk'])))
        res = {
            'id': self.rows[idx]['id'],
            'edited_headline_embedding': torch.stack(word_vectors),
            'orig_text': self.get_orig_text(row),
            'edited_text': headline,
            'orig': self.get_orig_phrase(row),
            'edit': row['edit']
        }
        if 'meanGrade' in self.rows[idx]:
            res['label'] = float(self.rows[idx]['meanGrade'])
            for k, v in self.rows[idx].items():
                if k.endswith('bin-label'):
                    res[k] = int(v)
        return res
      
      else:
        row = self.rows[idx]
        headline_1,headline_2 = self.get_edited_text(self.rows[idx])
        word_vectors_1 = []
        word_vectors_2 = []
        for word in headline_1.split():
            word_vectors_1.append(torch.from_numpy(self.glove_embeddings.get(word, self.glove_embeddings['unk'])))
        for word in headline_2.split():
            word_vectors_2.append(torch.from_numpy(self.glove_embeddings.get(word, self.glove_embeddings['unk'])))
        res = {
            'id': self.rows[idx]['id'],
            'edited_headline_embedding_1': torch.stack(word_vectors_1),
            'edited_headline_embedding_2': torch.stack(word_vectors_2),
            'orig_text': self.get_orig_text(row),
            'edited_text_1': headline_1,
            'edited_text_2': headline_2,
            'orig': self.get_orig_phrase(row),
            'edit_1': row['edit1'],
            'edit_2': row['edit2']
        }
        if 'meanGrade1' in self.rows[idx]:
            res['label1'] = float(self.rows[idx]['meanGrade1'])
            for k, v in self.rows[idx].items():
                if k.endswith('bin-label'):
                    res[k] = int(v)

        if 'meanGrade2' in self.rows[idx]:
            res['label2'] = float(self.rows[idx]['meanGrade2'])
            for k, v in self.rows[idx].items():
                if k.endswith('bin-label'):
                    res[k] = int(v)

        return res        


In [ ]:
def convert_sent(sentence):
        MAX_LEN = 80 
        sample_segments = []
        sentence = '[CLS] ' + sentence + ' [SEP]'
        tokens = tokenizer.encode(sentence, add_special_tokens=False)
        sample_segments.extend([0] * len(tokens))
        mask = [1] * len(tokens)
        mask[0] = 0  # [CLS]
        mask[-1] = 0  # [SEP]
        
        # padding
        input_ids = tokens
        if len(input_ids) > MAX_LEN:
            input_ids = input_ids[:MAX_LEN]
            mask = mask[:MAX_LEN]
            sample_segments = sample_segments[:MAX_LEN]
        else:
            pad = MAX_LEN - len(input_ids)
            input_ids.extend([0] * pad)
            sample_segments.extend([0] * pad)
            mask.extend([0] * pad)
        return torch.LongTensor(input_ids), torch.LongTensor(sample_segments), torch.LongTensor(mask)



In [ ]:
def BertDataLoader(dataset, batch_size, shuffle=True, pair=False,task='task-1'):
    indices = list(range(len(dataset)))
    if shuffle:
        random.shuffle(indices)
    if (task == 'task-1'):
      for i in range((len(dataset) + batch_size - 1) // batch_size):
          batch_indices = indices[i * batch_size: (i + 1) * batch_size]
          batch = [dataset[x] for x in batch_indices]
          xs = []
          segments = []
          masks = []
          ys = []

          for sample in batch:
              ids, sg, mask = convert_sent(sample['edited_text'])
              xs.append(ids)
              segments.append(sg)
              masks.append(mask)
              if 'label' in sample:
                  ys.append(sample['label'])
          xs = torch.stack(xs)
          segments = torch.stack(segments)
          masks = torch.stack(masks)
          ys = torch.Tensor(ys)
          yield xs, segments, masks, ys
    else:
      for i in range((len(dataset) + batch_size - 1) // batch_size):
          batch_indices = indices[i * batch_size: (i + 1) * batch_size]
          batch = [dataset[x] for x in batch_indices]
          xs = []
          segments = []
          masks = []
          ys = []

          for sample in batch:
              ids_1, sg_1, mask_1 = convert_sent(sample['edited_text_1'])
              ids_2, sg_2, mask_2 = convert_sent(sample['edited_text_2'])
              xs.append(ids_1)
              xs.append(ids_2)
              segments.append(sg_1)
              segments.append(sg_2)
              masks.append(mask_1)
              masks.append(mask_2)

              if 'label1' in sample:
                  ys.append(sample['label1'])
              if 'label2' in sample:
                  ys.append(sample['label2'])

          xs = torch.stack(xs)
          segments = torch.stack(segments)
          masks = torch.stack(masks)
          ys = torch.Tensor(ys)
          yield xs, segments, masks, ys



In [ ]:
class RMSELoss(torch.nn.Module):
    def __init__(self):
        super(RMSELoss,self).__init__()

    def forward(self,x,y):
        criterion = nn.MSELoss()
        loss = torch.sqrt(criterion(x, y))
        return loss


In [ ]:
class SavableModel(nn.Module):
    def __init__(self):
        super().__init__()

    def save(self, path):
        torch.save(self.state_dict(), path)

    def load(self, path):
        self.load_state_dict(torch.load(path))


In [ ]:
class BertRegressionModel(SavableModel):
    def __init__(self):
        super().__init__()
        self.hidden_size = 768
        self.bert = BertModel.from_pretrained('bert-base-cased')
        self.linear = nn.Linear(self.hidden_size, 1024)
        self.dropout = nn.Dropout(0.5)
        self.output = nn.Linear(1024, 1)

    def forward(self, x, segments, masks):
        x = self.bert(x, token_type_ids=segments, attention_mask=masks)[0]  # cls hidden state in the last layer
        x = torch.max(x, dim=-2)[0]
        x = torch.tanh(x)
        x = self.linear(x)
        x = nn.functional.relu(x)
        x = self.dropout(x)
        x = self.output(x)
        x = 3 * torch.sigmoid(x)  # normalize it to [0, 3]
        x = x.squeeze()
        return x



In [ ]:
def train(model: nn.Module):
    try:
        os.mkdir('checkpoints')
    except FileExistsError:
        pass
    try:
        os.mkdir('runs')
    except FileExistsError:
        pass

    TRAIN_LOG_FREQ = 10
    DEV_LOG_FREQ = 20

    train_writer = SummaryWriter('./runs/finetune_bert_train')
    dev_writer = SummaryWriter('./runs/finetune_bert_dev')
    train_dataset = HeadlineDataset('training')
    dev_dataset = HeadlineDataset('dev')
    optimizer = torch.optim.Adam(model.parameters())
    criterion = RMSELoss()

    steps = 0
    best_rmse = float('+inf')
    for epoch in range(30):
        running_loss = 0.0
        trainloader = BertDataLoader(train_dataset, 8, pair=False)
        dev_data = next(BertDataLoader(dev_dataset, len(dev_dataset), pair=False))
        for i, data in enumerate(trainloader):
            steps += len(data[0])
            model.train()
            # get the inputs; data is a list of [inputs, labels]
            inputs, segments, masks, labels = data
            inputs, segments, masks, labels = [x.to(device) for x in [inputs, segments, masks, labels]]

            # zero the parameter gradients
            optimizer.zero_grad()

            # forward + backward + optimize
            outputs = model(inputs, segments, masks)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()

            # print statistics
            running_loss += loss.item()

            if i % TRAIN_LOG_FREQ == TRAIN_LOG_FREQ - 1:
                print('[%d, %5d]     loss: %.6f' %
                      (epoch + 1, i + 1, running_loss / TRAIN_LOG_FREQ))
                train_writer.add_scalar('epoch', epoch + 1, steps)
                train_writer.add_scalar('loss', running_loss / TRAIN_LOG_FREQ, steps)
                running_loss = 0.0
            if i % DEV_LOG_FREQ == DEV_LOG_FREQ - 1: 
                model.eval()
                dev_inputs, dev_segments, dev_masks, dev_labels = [x.to(device) for x in dev_data]
                val_loss = criterion(outputs, labels)
                print('[%d, %5d]     val loss: %.6f' %
                        (epoch + 1, i + 1, val_loss))
                dev_writer.add_scalar('loss', val_loss, steps)
                # save
                model.save('./checkpoints/bert_last.pt')
                if val_loss < best_rmse:
                    best_rmse = val_loss
                    copy2('./checkpoints/bert_last.pt', 'checkpoints/bert_best.pt')
        # validation
        model.eval()
        dev_inputs, dev_segments, dev_masks, dev_labels = [x.to(device) for x in dev_data]
        val_loss = criterion(outputs, labels)
        print('[%d]      val loss: %.6f' %
                (epoch + 1, val_loss))
        dev_writer.add_scalar('loss', val_loss, steps)
        # save
        model.save('./checkpoints/bert_last.pt')
        if val_loss < best_rmse:
            best_rmse = val_loss
            copy2('./checkpoints/bert_last.pt', 'checkpoints/bert_best.pt')



In [ ]:
def predict(model):
    from csv import writer
    test_dataset_1 = HeadlineDataset('test')
    test_loader_1 = BertDataLoader(test_dataset_1, batch_size=64)
    test_dataset_2 = HeadlineDataset('test',task = 'task-2')
    test_loader_2 = BertDataLoader(test_dataset_2, batch_size=64,task = 'task-2')
    model.load('./checkpoints/bert_best.pt')
    
    y_pred_1 = []
    y_pred_2 = []

    for data in test_loader_1:
        inputs, segments, masks, _ = [x.to(device) for x in data]
        ys = model(inputs, segments, masks).cpu().tolist()
        y_pred_1.extend(ys)
    
    for data in test_loader_2:
        inputs, segments, masks, _ = [x.to(device) for x in data]
        ys = model(inputs, segments, masks).cpu().tolist()
        y_pred_2.extend(ys)
    
    y_out_2=[]
    for i in range( 0 , len(y_pred_2)-1 , 2 ):
      if (y_pred_2[i] == y_pred_2[i+1]):
         y_out_2.append(0)

      elif (y_pred_2[i] > y_pred_2[i+1]):
         y_out_2.append(1)

      else:
         y_out_2.append(2)
        


    with open('./data/task-1-output.csv', 'w') as f:
        output_writer = writer(f)
        output_writer.writerow(('id', 'pred'))
        for row, pred in zip(test_dataset_1, y_pred_1):
            output_writer.writerow((row['id'], pred))
    
    with open('./data/task-2-output.csv', 'w') as f:
        output_writer = writer(f)
        output_writer.writerow(('id', 'pred'))
        for row, pred in zip(test_dataset_2, y_out_2):
            output_writer.writerow((row['id'], pred))

    os.chdir('data')
    subprocess.run(['zip', 'task-1-output.zip', 'task-1-output.csv'])
    subprocess.run(['zip', 'task-2-output.zip', 'task-2-output.csv'])
    os.chdir('..')


In [ ]:
model = BertRegressionModel()
model = model.to(device)



In [ ]:
train(model)

Streaming output truncated to the last 5000 lines.
[3,   470]     loss: 0.572422
[3,   480]     loss: 0.551836
[3,   480]     val loss: 0.661309
[3,   490]     loss: 0.621501
[3,   500]     loss: 0.584512
[3,   500]     val loss: 0.602390
[3,   510]     loss: 0.551032
[3,   520]     loss: 0.570916
[3,   520]     val loss: 0.686654
[3,   530]     loss: 0.546741
[3,   540]     loss: 0.566618
[3,   540]     val loss: 0.423971
[3,   550]     loss: 0.572129
[3,   560]     loss: 0.580848
[3,   560]     val loss: 0.724820
[3,   570]     loss: 0.522976
[3,   580]     loss: 0.626317
[3,   580]     val loss: 0.961698
[3,   590]     loss: 0.603800
[3,   600]     loss: 0.614172
[3,   600]     val loss: 0.624668
[3,   610]     loss: 0.505347
[3,   620]     loss: 0.658877
[3,   620]     val loss: 0.671658
[3,   630]     loss: 0.581135
[3,   640]     loss: 0.611163
[3,   640]     val loss: 0.664622
[3,   650]     loss: 0.593454
[3,   660]     loss: 0.607171
[3,   660]     val loss: 0.377387
[3,   670

In [ ]:
predict(model)